In [1]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [10]:
import dotenv

dotenv.load_dotenv()
import os

url = os.getenv("COSMOS_DB_URL")
key = os.getenv("COSMOS_DB_KEY")


In [ ]:
from azure.cosmos import CosmosClient

# Replace with your account URL and key
url = url
key = key

client = CosmosClient(url, credential=key)

# Get the database and container clients
database = client.get_database_client("aidb")
container = database.get_container_client("ai_rag_cnt")

### Explanation:
- path → field holding your vector.
- type → algorithm for similarity search (quantizedFlat recommended).
- dataType → usually float32 for embeddings.
- dimensions → embedding size (e.g., OpenAI text-embedding-ada-002 = 1536).

In [3]:
document = {
    "id": "doc1",  # unique id
    "content": "Azure Cosmos DB vector search example",
    "embedding": [
        0.12, 0.98, 0.34, 0.56, 0.78, 0.11, 0.45, 0.66
        # … full vector length matching your model, e.g., 1536
    ]
}

In [4]:
container.create_item(document)
print("Document inserted successfully!")

Document inserted successfully!


In [5]:
for item in container.read_all_items():
    print(item)

{'id': 'doc1', 'content': 'Azure Cosmos DB vector search example', 'embedding': [0.12, 0.98, 0.34, 0.56, 0.78, 0.11, 0.45, 0.66], '_rid': 'deVRANDLpNoBAAAAAAAAAA==', '_self': 'dbs/deVRAA==/colls/deVRANDLpNo=/docs/deVRANDLpNoBAAAAAAAAAA==/', '_etag': '"77017eba-0000-0700-0000-68c56ab80000"', '_attachments': 'attachments/', '_ts': 1757768376}


In [ ]:
from azure.cosmos import CosmosClient, PartitionKey

# -----------------------------
# 1️⃣ Setup Cosmos DB client
# -----------------------------
url = url
key = key
database_name = "aidb"
container_name = "ai_rag_cnt"

client = CosmosClient(url, credential=key)
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# -----------------------------
# 2️⃣ Insert sample document
# -----------------------------
doc = {
    "id": "doc1",
    "content": "Azure Cosmos DB vector search example",
    "embedding": [
        0.12, 0.98, 0.34, 0.56, 0.78, 0.11, 0.45, 0.66
        # ... your full vector, e.g., 1536-dim for OpenAI embeddings
    ]
}

# Insert the document (ignore if already exists)
try:
    container.create_item(doc)
    print("Document inserted successfully!")
except Exception as e:
    print("Document may already exist:", e)

# -----------------------------
# 3️⃣ Define query embedding
# -----------------------------
query_embedding = [
    0.11, 0.97, 0.33, 0.55, 0.77, 0.12, 0.44, 0.65
    # ... must match dimension of vector index
]

# -----------------------------
# 4️⃣ Query top-5 similar vectors
# -----------------------------
query = """
SELECT c.id, c.content
FROM c
ORDER BY VECTOR_DISTANCE(c.embedding, @vec) ASC
OFFSET 0 LIMIT 5
"""

parameters = [{"name": "@vec", "value": query_embedding}]

try:
    results = list(container.query_items(
        query=query,
        parameters=parameters,
        enable_cross_partition_query=True
    ))

    print("Top-5 similar documents:")
    for item in results:
        print(item)

except Exception as e:
    print("Query failed. Make sure vector search is enabled:", e)

Document may already exist: (Conflict) Entity with the specified id already exists in the system., 
RequestStartTime: 2025-09-13T13:18:26.3991245Z, RequestEndTime: 2025-09-13T13:18:26.4012793Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2025-09-13T13:17:30.2944270Z","cpu":0.474,"memory":435614216.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.1578,"availableThreads":32765,"minThreads":104,"maxThreads":32767},"numberOfOpenTcpConnection":537},{"dateUtc":"2025-09-13T13:17:40.3046828Z","cpu":0.759,"memory":435593816.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0656,"availableThreads":32765,"minThreads":104,"maxThreads":32767},"numberOfOpenTcpConnection":537},{"dateUtc":"2025-09-13T13:17:50.3147570Z","cpu":1.171,"memory":435594788.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.092,"availableThreads":32763,"minThreads":104,"maxThreads":32767},"numberOfOpenTcpConnection":534},{"dateUtc":"2025-09-13T